<a href="https://colab.research.google.com/github/BartoszJanJerzy/M-L-Bootcamp-cz.1/blob/master/16_metryki_klasyfikacji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

#Metryki - klasyfikacja binarna

##Accuracy - dokładność

In [2]:
y_true = np.array([1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1])
y_pred = np.array([0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1])

In [4]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_true, y_pred)
acc

0.8

In [15]:
results = pd.DataFrame({'y_true':y_true, 'y_pred':y_pred})
results.head()

,y_true,y_pred
0,1,0
1,0,0
2,1,1
3,1,1
4,0,0


In [17]:
results.sort_values(by='y_true', inplace=True)
results.reset_index(drop=True, inplace=True)
results['sample'] = results.index + 1
results.head()

,y_true,y_pred,sample
0,0,0,1
1,0,1,2
2,0,0,3
3,0,0,4
4,0,0,5


In [18]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_true'], mode='markers', name='y_true'), row=1, col=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_pred'], mode='markers', name='y_pred'), row=2, col=1)
fig.update_layout(width=1000, height=500, title='Klasyfikator binarny')
fig.show()

##Macierz konfuzji/pomyłek

In [34]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)
print(cm[::-1])

[[ 8  2]
 [ 4 16]]
[[ 4 16]
 [ 8  2]]


In [26]:
import plotly.figure_factory as ff

def plot_confusion_matrix(cm):
  cm = cm[::-1] #odwrócenie cm 'do góry nogami'
  cm = pd.DataFrame(cm, columns=['pred_0','pred_1'], index=['true_1','true_0']) 

  print(cm)

  fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index),
                                    colorscale='ice', showscale=True, reversescale=True)
  fig.update_layout(width=500, height=500, title='Confusion Matrix', font_size=16)
  fig.show()

plot_confusion_matrix(cm)


        pred_0  pred_1
true_1       4      16
true_0       8       2


##Charakterystyki macierzy konfuzji

In [35]:
tp, fp, fn, tn = cm.ravel()

In [36]:
# False Positive Rate - Type I Error
fpr = fp / (fp + tn)
fpr

0.1111111111111111

In [37]:
# False Negative Rate - Type II Error
fnr = fn / (fn + tp)
fnr

0.3333333333333333

In [38]:
# Precision - ile obserwacji przewidzianych jako pozytywne, rzeczywiście są pozytywne
precision = tp / (tp + fp)
precision

0.8

In [39]:
# recall - jaki odsetek obserwacji pozytywnych udało się złapać
recall = tp / (tp + fn)
recall

0.6666666666666666

In [40]:
# raport klasyfikacji
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.80      0.73        10
           1       0.89      0.80      0.84        20

    accuracy                           0.80        30
   macro avg       0.78      0.80      0.78        30
weighted avg       0.81      0.80      0.80        30



##Krzywa ROC

In [43]:
from sklearn.metrics import roc_curve

fpr, tpr, tresh = roc_curve(y_true, y_pred, pos_label=1)

roc = pd.DataFrame({'fpr':fpr, 'tpr':tpr})
roc

,fpr,tpr
0,0.0,0.0
1,0.2,0.8
2,1.0,1.0


In [44]:
def plot_roc_curve(y_true, y_pred):
  # Binary classification
  
  # import krzywej roc
  from sklearn.metrics import roc_curve
  fpr, tpr, tresh = roc_curve(y_true, y_pred, pos_label=1)

  # tworzenie wykresu
  fig = go.Figure(
      data=[
            go.Scatter(x=roc['fpr'], y=roc['tpr'],
                       line_color='red',
                       name='ROC Curve'),
            go.Scatter(x=[0,1], y=[0,1],
                       mode='lines',
                       line_dash='dash',
                       line_color='navy')
      ],
      layout=go.Layout(xaxis_title='False Positive Rate',
                       yaxis_title='True Positive Rate',
                       title='ROC Curve',
                       showlegend=False,
                       width=800)
  )

  fig.show()

In [46]:
plot_roc_curve(y_true,y_pred)

#Klasyfikacja wieloklasowa

In [48]:
y_true = np.array([1, 0, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2])
y_pred = np.array([0, 0, 1, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0, 1, 1, 1, 2, 2])

from sklearn.metrics import accuracy_score

accuracy_score(y_true, y_pred)

0.7241379310344828

##Macierz konfuzji

In [49]:
cm = confusion_matrix(y_true, y_pred)
cm

array([[ 6,  1,  0],
       [ 3, 10,  2],
       [ 0,  2,  5]])

In [50]:
def plot_confusion_matrix(cm):
  cm = cm[::-1] #odwrócenie cm 'do góry nogami'
  cm = pd.DataFrame(cm, columns=['pred_0','pred_1', 'pred_2'], index=['true_2','true_1','true_0']) 

  print(cm)

  fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index),
                                    colorscale='ice', showscale=True, reversescale=True)
  fig.update_layout(width=500, height=500, title='Confusion Matrix', font_size=16)
  fig.show()

plot_confusion_matrix(cm)

        pred_0  pred_1  pred_2
true_2       0       2       5
true_1       3      10       2
true_0       6       1       0


##Raport klasyfikacji

In [51]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=['label_1','label_2','label_3']))

              precision    recall  f1-score   support

     label_1       0.67      0.86      0.75         7
     label_2       0.77      0.67      0.71        15
     label_3       0.71      0.71      0.71         7

    accuracy                           0.72        29
   macro avg       0.72      0.75      0.73        29
weighted avg       0.73      0.72      0.72        29

